In [96]:
load("GeneralizedShallowWaterVariables.sage")
load('../Sage/VectorMatrixVariables.sage')
num_moments = 1
set_generalized_shallow_water_variables(num_moments)
num_basis_cpts = 2 + num_moments

s = var('s', domain="positive")
# Local Lax Friedrichs
c = -s
d = s
I = matrix.identity(num_basis_cpts)
B = block_matrix([[-Q_q, I],[-c*d*I, (c + d)*I]])
print(B)

[       0        0        0|       1        0        0]
[       0        0        0|       0        1        0]
[       0        0 -q_2/q_1|       0        0        1]
[--------------------------+--------------------------]
[     s^2        0        0|       0        0        0]
[       0      s^2        0|       0        0        0]
[       0        0      s^2|       0        0        0]


In [97]:
eig = B.eigenvectors_right()
eigenvalues = flatten([[e[0].simplify_full() for i in range(e[2])] for e in eig])
eigenvectors = flatten([[ev.simplify_full() for ev in e[1]] for e in eig])
R = matrix(eigenvectors).T
L = R.inverse().simplify_full()
print(eigenvalues)

[-1/2*(q_2 + sqrt(4*q_1^2*s^2 + q_2^2))/q_1, -1/2*(q_2 - sqrt(4*q_1^2*s^2 + q_2^2))/q_1, -s, -s, s, s]


In [98]:
Q_r = get_vector_variable("Q_r", num_basis_cpts)
Q_l = get_vector_variable("Q_l", num_basis_cpts)
f_Q_r = get_vector_variable("f_Q_r", num_basis_cpts)
f_Q_l = get_vector_variable("f_Q_l", num_basis_cpts)
Q_diff = Q_r - Q_l
f_diff = f_Q_r - f_Q_l
v = vector(Q_diff.list() + f_diff.list())
alphas = L*v

In [99]:
(R*alphas - v).simplify_full()

(0, 0, 0, 0, 0, 0)

In [100]:
# wave propogation
Am_deltaQ = sum([(min(eigenvalues[i], 0)*alphas[i]*eigenvectors[i][:num_basis_cpts]).simplify_full() for i in range(2*num_basis_cpts)])
print(Am_deltaQ)
Ap_deltaQ = sum([(max(eigenvalues[i], 0)*alphas[i]*eigenvectors[i][:num_basis_cpts]).simplify_full() for i in range(2*num_basis_cpts)])
print(Ap_deltaQ)

(1/2*(Q_l_0 - Q_r_0)*s - 1/2*f_Q_l_0 + 1/2*f_Q_r_0, 1/2*(Q_l_1 - Q_r_1)*s - 1/2*f_Q_l_1 + 1/2*f_Q_r_1, -1/2*((f_Q_l_2 - f_Q_r_2)*q_1*q_2^2 - (Q_l_2 - Q_r_2)*q_2^3 + 4*((f_Q_l_2 - f_Q_r_2)*q_1^3 - (Q_l_2 - Q_r_2)*q_1^2*q_2)*s^2 + (2*(Q_l_2 - Q_r_2)*q_1^2*s^2 - (f_Q_l_2 - f_Q_r_2)*q_1*q_2 + (Q_l_2 - Q_r_2)*q_2^2)*sqrt(4*q_1^2*s^2 + q_2^2))/(4*q_1^3*s^2 + q_1*q_2^2) - 1/2*((f_Q_l_2 - f_Q_r_2)*q_1*q_2^2 - (Q_l_2 - Q_r_2)*q_2^3 + 4*((f_Q_l_2 - f_Q_r_2)*q_1^3 - (Q_l_2 - Q_r_2)*q_1^2*q_2)*s^2 - (2*(Q_l_2 - Q_r_2)*q_1^2*s^2 - (f_Q_l_2 - f_Q_r_2)*q_1*q_2 + (Q_l_2 - Q_r_2)*q_2^2)*sqrt(4*q_1^2*s^2 + q_2^2))/(4*q_1^3*s^2 + q_1*q_2^2))
(-1/2*(Q_l_0 - Q_r_0)*s - 1/2*f_Q_l_0 + 1/2*f_Q_r_0, -1/2*(Q_l_1 - Q_r_1)*s - 1/2*f_Q_l_1 + 1/2*f_Q_r_1, -1/2*((f_Q_l_2 - f_Q_r_2)*q_1*q_2^2 - (Q_l_2 - Q_r_2)*q_2^3 + 4*((f_Q_l_2 - f_Q_r_2)*q_1^3 - (Q_l_2 - Q_r_2)*q_1^2*q_2)*s^2 + (2*(Q_l_2 - Q_r_2)*q_1^2*s^2 - (f_Q_l_2 - f_Q_r_2)*q_1*q_2 + (Q_l_2 - Q_r_2)*q_2^2)*sqrt(4*q_1^2*s^2 + q_2^2))/(4*q_1^3*s^2 + q_1*q_2^2) -

In [106]:
# numerical flux
F_l = (f_Q_l + Am_deltaQ).simplify_full()
print(F_l)
F_r = (f_Q_r - Ap_deltaQ).simplify_full()
print(F_r)

(1/2*(Q_l_0 - Q_r_0)*s + 1/2*f_Q_l_0 + 1/2*f_Q_r_0, 1/2*(Q_l_1 - Q_r_1)*s + 1/2*f_Q_l_1 + 1/2*f_Q_r_1, (f_Q_r_2*q_1 + (Q_l_2 - Q_r_2)*q_2)/q_1)
(1/2*(Q_l_0 - Q_r_0)*s + 1/2*f_Q_l_0 + 1/2*f_Q_r_0, 1/2*(Q_l_1 - Q_r_1)*s + 1/2*f_Q_l_1 + 1/2*f_Q_r_1, (f_Q_l_2*q_1 - (Q_l_2 - Q_r_2)*q_2)/q_1)


In [110]:
(((f_Q_r_2*q_1 + (Q_l_2 - Q_r_2)*q_2)/q_1) - ((f_Q_r_2 + (Q_l_2 - Q_r_2)*q_2/q_1))).simplify_full()

0

In [132]:
eigenvalues[0].subs(q_to_p).simplify_full()

-1/2*(h*u + sqrt(4*h^2*s^2 + h^2*u^2))/h

In [129]:
-1/2*(q_2 + sqrt(4*q_1^2*s^2 + q_2^2))/q_1

-1/2*(q_2 + sqrt(4*q_1^2*s^2 + q_2^2))/q_1